In [ ]:
import pandas as pd

In [9]:
file_path = 'data.csv'
df = pd.read_csv('data.csv')

print(df.head())
print(df.columns)

    Rk  Gcar  Gtm        Date Team Unnamed: 5  Opp          Result GS     MP  \
0  1.0   1.0  1.0  2006-04-22  CLE        NaN  WAS         W 97-86  *  48:00   
1  2.0   2.0  2.0  2006-04-25  CLE        NaN  WAS         L 84-89  *  43:28   
2  3.0   3.0  3.0  2006-04-28  CLE          @  WAS         W 97-96  *  47:37   
3  4.0   4.0  4.0  2006-04-30  CLE          @  WAS        L 96-106  *  45:20   
4  5.0   5.0  5.0  2006-05-03  CLE        NaN  WAS  W 121-120 (OT)  *  46:07   

   ...  DRB   TRB   AST  STL  BLK   TOV   PF   PTS  GmSc   +/-  
0  ...  8.0  11.0  11.0  0.0  0.0   4.0  3.0  32.0  23.3  11.0  
1  ...  7.0   9.0   2.0  2.0  3.0  10.0  5.0  26.0   6.7  -2.0  
2  ...  4.0   5.0   3.0  2.0  0.0   4.0  3.0  41.0  27.4   1.0  
3  ...  5.0   6.0   5.0  0.0  0.0   7.0  5.0  38.0  23.0 -16.0  
4  ...  2.0   7.0   6.0  2.0  0.0   4.0  5.0  45.0  38.4  -4.0  

[5 rows x 34 columns]
Index(['Rk', 'Gcar', 'Gtm', 'Date', 'Team', 'Unnamed: 5', 'Opp', 'Result',
       'GS', 'MP', 'FG', 'FGA',

In [12]:
import requests
from lxml import html

# Load the page
url = "https://www.asapsports.com/show_player.php?id=13888"
response = requests.get(url)
tree = html.fromstring(response.content)

# Get all interview links from the 3rd <td> in each row of the main interview table
interview_links = tree.xpath('//table[1]//tr/td[3]/b/a')

interviews = []
for link in interview_links:
    href = link.get('href')
    text = link.text_content().strip()
    if href:
        full_url = f"https://www.asapsports.com/{href}"
        interviews.append({'title': text, 'url': full_url})

# Print all links
for interview in interviews:
    print(f"{interview['title']}: {interview['url']}")


NBA ALL-STAR GAME: https://www.asapsports.com/http://www.asapsports.com/show_interview.php?id=204675
ALL-STAR GAME: https://www.asapsports.com/http://www.asapsports.com/show_interview.php?id=194257
ALL-STAR GAME: https://www.asapsports.com/http://www.asapsports.com/show_interview.php?id=194258
NBA IN-SEASON TOURNAMENT: https://www.asapsports.com/http://www.asapsports.com/show_interview.php?id=193132
NBA IN-SEASON TOURNAMENT: https://www.asapsports.com/http://www.asapsports.com/show_interview.php?id=193098
NBA IN-SEASON TOURNAMENT: https://www.asapsports.com/http://www.asapsports.com/show_interview.php?id=193082
NBA WESTERN CONFERENCE FINALS: LAKERS VS. NUGGETS: https://www.asapsports.com/http://www.asapsports.com/show_interview.php?id=187284
NBA WESTERN CONFERENCE FINALS: LAKERS VS. NUGGETS: https://www.asapsports.com/http://www.asapsports.com/show_interview.php?id=187224
NBA WESTERN CONFERENCE FINALS: LAKERS VS. NUGGETS: https://www.asapsports.com/http://www.asapsports.com/show_interv

In [15]:
import requests
from lxml import html
import time
import json

# Step 1: Collect all interview links from the player page
main_url = "https://www.asapsports.com/show_player.php?id=13888"
response = requests.get(main_url)
tree = html.fromstring(response.content)

# Extract the first 5 interview links
raw_links = tree.xpath('//table[1]//tr/td[3]/b/a')
interviews = []
for link in raw_links[:5]:
    href = link.get('href')
    title = link.text_content().strip()
    if href:
        # normalize URL in case it's missing the leading slash
        full_url = f"https://www.asapsports.com/{href.lstrip('/')}"
        interviews.append({'title': title, 'url': full_url})

# Step 2: Visit each URL to fetch date and Q&A paragraphs
all_transcripts = []

for interview in interviews:
    try:
        res = requests.get(interview['url'])
        page = html.fromstring(res.content)

        # Extract the date from the <h2>
        date_elem = page.xpath('//h2/text()')
        date = date_elem[0].strip() if date_elem else 'Unknown Date'

        # Extract all <p> tags as Q&A
        qa_paragraphs = [
            p.text_content().strip()
            for p in page.xpath('//p')
            if p.text_content().strip()
        ]

        all_transcripts.append({
            'title': interview['title'],
            'url': interview['url'],
            'date': date,
            'qa': qa_paragraphs
        })

        # Be polite to the server
        time.sleep(1)

    except Exception as e:
        print(f"❌ Failed to process {interview['url']}: {e}")

# Step 3: Save your first‑5 output to JSON for verification
with open("lebron_interviews_first5.json", "w", encoding="utf-8") as f:
    json.dump(all_transcripts, f, indent=2, ensure_ascii=False)

print("✅ Done—check lebron_interviews_first5.json")


✅ Done—check lebron_interviews_first5.json
